In [1]:
import os
import shutil

root_train_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/train"
root_test_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/test"
root_validation_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/validation"
root_visualizaion_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/visualization"
saved_model_file_name = "currency_detector_mobilenet.h5"
tensorboard_dir = "/home/xiaoyzhu/notebooks/currency_detector/data/tensorboard"

In [5]:

from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224,224
train_data_dir = root_train_dir
validation_data_dir = root_validation_dir
nb_train_samples = 1672
nb_validation_samples = 560
train_steps = 100 # 1672 training samples/batch size of 32 = 52 steps. We are doing heavy data processing so put 500 here
validation_steps = 20 # 560 validation samples/batch size of 32 = 10 steps. We put 20 for validation steps
batch_size = 32
epochs = 100

def build_model():
    # constructing the model
    model = applications.mobilenet.MobileNet(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3),
                                  pooling='avg')

    # only train the last 2 layers
    for layer in model.layers[:-10]:
        layer.trainable = False

    # Adding custom Layers
    x = model.output
    # x = Flatten()(x)
    predictions = Dense(14, activation="softmax")(x)

    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)
    
    return model_final

model_final = build_model()
# compile the model
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.001), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="nearest",
    zoom_range=0.3,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    rotation_range=180)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="nearest",
    zoom_range=0.3,
    rotation_range=30)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    # save_to_dir = root_visualizaion_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

# Save the model according to the conditions
checkpoint = ModelCheckpoint("currency_detector_test.h5", monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False,
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# Train the model
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_steps,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = validation_steps,
    workers=16,
    callbacks=[checkpoint, early])

Found 2092 images belonging to 14 classes.
Found 420 images belonging to 14 classes.
Epoch 1/100
100/100 [==============================] - 27s 270ms/step - loss: 0.9529 - acc: 0.6958 - val_loss: 2.7200 - val_acc: 0.3922

Epoch 00001: val_loss improved from inf to 2.72004, saving model to currency_detector_test.h5
Epoch 2/100
100/100 [==============================] - 22s 220ms/step - loss: 0.2925 - acc: 0.9098 - val_loss: 2.1659 - val_acc: 0.5637

Epoch 00002: val_loss improved from 2.72004 to 2.16590, saving model to currency_detector_test.h5
Epoch 3/100
100/100 [==============================] - 23s 231ms/step - loss: 0.2058 - acc: 0.9326 - val_loss: 2.3575 - val_acc: 0.4623

Epoch 00003: val_loss did not improve
Epoch 4/100
100/100 [==============================] - 22s 218ms/step - loss: 0.1653 - acc: 0.9453 - val_loss: 2.5877 - val_acc: 0.4641

Epoch 00004: val_loss did not improve
Epoch 5/100
100/100 [==============================] - 23s 233ms/step - loss: 0.1403 - acc: 0.9509 

In [ ]:

from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224,224
image_scale = 1./255 # or 1./255 if you want to rescale (which should be the case)

nb_train_samples = 1672
nb_validation_samples = 560
train_steps = 100 # 1672 training samples/batch size of 32 = 52 steps. We are doing heavy data processing so put 500 here
validation_steps = 20 # 560 validation samples/batch size of 32 = 10 steps. We put 20 for validation steps
batch_size = 32
epochs = 100

def build_model():
    # constructing the model
    model = applications.mobilenet.MobileNet(weights="imagenet", include_top=False, 
                                             input_shape=(img_width, img_height, 3), pooling='avg')
    # only train the last 2 layers
    for layer in model.layers[:-10]:
        layer.trainable = False
    # Adding custom Layers
    x = model.output
    predictions = Dense(14, activation="softmax")(x)
    # creating the final model
    model_final = Model(inputs=model.input, outputs=predictions)
    
    return model_final

model_final = build_model()
# compile the model
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.001), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale=image_scale,
    fill_mode="nearest",
    zoom_range=0.3,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    rotation_range=360)

validation_datagen = ImageDataGenerator(
    rescale=image_scale,
    fill_mode="nearest",
    zoom_range=0.3,
    rotation_range=30)

test_datagen = ImageDataGenerator(
    rescale=image_scale,
    fill_mode="nearest",
    zoom_range=0.3,
    rotation_range=30)

train_generator = train_datagen.flow_from_directory(
    root_train_dir,
    # save_to_dir = root_visualizaion_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = validation_datagen.flow_from_directory(
    root_validation_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    root_test_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

# Class index
print("training labels are:", validation_generator.class_indices)

# Save the model according to the conditions
checkpoint = ModelCheckpoint(saved_model_file_name, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False,
                             mode='auto', period=1)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

tensorboard = TensorBoard(log_dir = tensorboard_dir)
# Train the model
history = model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_steps,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = validation_steps,
    workers=16,
    callbacks=[checkpoint, earlystopping, tensorboard])



# Test the model and generate confusion matrix

In [ ]:
probabilities = model_final.predict_generator(test_generator, workers = 16, verbose = 1)




In [ ]:
import numpy as np

print(test_generator.class_indices)
y_true = ((test_generator.classes))
y_pred = (np.argmax(probabilities, axis = 1))

from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
np.set_printoptions(precision=2)

name = ['5_dollar_back', '2_dollar_back', '1_dollar_back', '2_dollar_front', '20_dollar_front', '10_dollar_back', '50_dollar_back', '100_dollar_front', '1_dollar_front', '50_dollar_front', '20_dollar_back', '5_dollar_front', '10_dollar_front', '100_dollar_back']

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confusion_matrix, classes=name,
                      title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(confusion_matrix, classes=name, normalize=True,
#                       title='Normalized confusion matrix')

plt.show()

In [ ]:
from sklearn import svm, datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names
print(class_names)

In [ ]:
# dir(test_generator)
y_true = validation_generator.classes 


y_true = np.array([0] * 1000 + [1] * 1000)
y_pred = probabilities > 0.5



# Convert models to ONNX for further use

In [ ]:
import onnxmltools
import coremltools

# install from https://github.com/onnx/onnxmltools and https://github.com/apple/coremltools

model_coreml = coremltools.converters.keras.convert(saved_model_file_name, image_scale = image_scale)
model_onnx = onnxmltools.convert.convert_coreml(model_coreml, "currency_detector")

# Save as protobuf
onnxmltools.utils.save_model(model_onnx, saved_model_file_name + ".onnx")

# Utils to Move data around

In [ ]:
# utility to move data around
def move_files_subfolders(root_src_dir, root_target_dir, operation, image_number):
    for src_dir, dirs, files in os.walk(root_src_dir):
        num_temp = 0
        dst_dir = src_dir.replace(root_src_dir, root_target_dir)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        for individual_file in files:
            if num_temp < image_number:
                src_file = os.path.join(src_dir, individual_file)
                dst_file = os.path.join(dst_dir, individual_file)
                if os.path.exists(dst_file):
                    os.remove(dst_file)
                if operation is 'copy':
                    shutil.copy(src_file, dst_dir)
                elif operation is 'move':
                    shutil.move(src_file, dst_dir)
                num_temp += 1
            else:
                break

# move_files_subfolders(root_train_dir, root_validation_dir,  'move', 10)
